## Projeto para Disciplina de Mineração de Dados  
## Aplicação de Técnica de Classificação para Solução de Captcha  Sonoro
### Programa de Pós Graduação em Computação UFABC - 2º Quadrimestre de 2018
#### Integrantes:
Erick Fernandes da Cruz <br>
Gustavo Borges Lugoboni


### 1 - Introdução
Segundo o artigo de Gantz (2012) o crescimento na quantidade de dados ocorre de forma exponencial e até 2020 poderemos chegar a quantidade de 40 trilhões de Gigabytes de dados, e neste contexto as técnicas de Mineração de Dados (Data Mining) apresentam ferramentas para encontrarmos relações desconhecidas e para sumarizar os dados em formas que sejam compreensíveis e úteis, o que dificilmente seria feito sem a utilização de técnicas adequadas.

Durante a disciplina foram estudadas diversas técnicas utilizadas em Mineração de Dados, algoritmos de regressão e classificação, porém o presente projeto se limita ao estudo dos algoritmos de classificação. Os problemas de classificação tem como objetivo principal identificar qual a classe mais que um novo dado deve ser classificado.

Neste trabalho utilizaremos algoritmos de classificação para classificar de forma adequada um conjunto de arquivos wav que possuem em 4 caracteres sendo falados por arquivo, onde os possíveis caracteres são a,b,c,d,h,m,n,x,6,7. Para isso teremos um conjunto de arquivos treinamento e um conjunto de arquivos de validação. Nosso objetivo é através da utilização de um algoritmo de classificação conseguir inferir corretamente as letras que estão contidas em cada um dos arquivos wav do conjunto de validação.

Para o desenvolvimento do projeto foi utilizada a linguagem Python (Versão 3.5.2) e para auxiliar o desenvolvimento utilizamos algumas bibliotecas para auxiliar no desenvolvimento. Dentre essa bibliotecas utilizamos a Numpy e Scipy que auxilia em vários cálculos realizados e manipulações de Arrays, a biblioteca Librosa que apresenta ferramentas para manipulação de arquivos de aúdio, a bilioteca Sklearn que possui um grande número de técnicas de Mineração de Dados que foram aproveitadas neste projeto e a bilioteca Bokeh que possui funções que auxiliaram na geração dos histogramas que utilizamos em nossa análise.

No trecho de código a seguir é possível realizar as importações das bibliotecas que serão utilizadas no decorrer do projeto.

In [71]:
#Bibliotecas utilizadas

import copy
import os

import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from bokeh.io import output_notebook
from bokeh.layouts import gridplot
from bokeh.models import CDSView, ColumnDataSource, GroupFilter
from bokeh.plotting import figure, show
from sklearn import preprocessing, tree
from sklearn.cluster import AgglomerativeClustering, KMeans
from sklearn.datasets import make_classification
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier

### 2 - Análise Exploratória
A base de dados estudada possuía alguns problemas que necessitaram de certas transformações para facilitar a obtenção dos atributos. A primeira etapa segmentar o arquivo wav de forma a obter 4 partes correspondentes a cada carácter do arquivo. Para isso inicialmente separamos o arquivo em 4 partes iguais, visto que a duração dos arquivos de áudio que compunham o captcha possuíam tamanhos iguais.

Após esta separação, a próximo etapa foi separar os trechos onde apenas continha ruído de fundo do trecho que o carácter é falada. Para isto segmentamos o áudio em pequenos subconjuntos e utilizamos o pico de frequência do primeiro subconjunto mais o desvio padrão como frequência de corte, considerando que sempre o primeiro subconjunto será um trecho com apenas ruído. Utilizamos esta frequência de corte para verificar os subconjuntos que possuíam um frequência superior a ela e unimos os conjuntos com estas características que estivessem a uma distancia de no máximo 2 subconjuntos. Com os subconjuntos unidos verificamos qual a união de subconjuntos possuía o maior número de elementos e consideramos esta como a parte correspondente ao carácter.

Após identificarmos o exato pedaço correspondente de cada carácter, a transformação seguinte teve como objetivo remover o ruído de fundo que estava presente no próprio carácter. Para isso subtraímos da região correspondente as frequências dos caracteres por uma frequência de corte correspondente ao ruído de fundo de cada áudio.

Por fim verificamos que devido a diferença de volume dos áudios, apesar de um mesmo carácter falado por duas pessoas diferentes possuir uma curva de frequência semelhante, os valores referentes a frequência se mostravam bem distintos. Para contornamos este problema normalizamos os valores de frequência correspondentes a cada carácter entre -1 e 1.

A seguir é possível verificar o código utilizado para a realização das manipulações citadas acima, sendo este primeiro trecho corresponde aos Arrays que foram criados para obter respectivamente a onda e ao char de cada carácter do conjunto de treinamento.

In [72]:
WaveLetters = list()
Y = list()

A função a seguir recebe um Array com os valores de frequências correspondentes a cada carácter e devolve como saída o mesmo Array com os valores de frequência normalizados entre -1 e 1.

In [73]:
def normalizar(data) :
    
    #Definir valor máximo
    maximo = 0;
    if (max(data) > np.absolute(min(data))):
        maximo = max(data)
    else: 
        maximo = np.absolute(min(data))
    
    #Normalizar vetor entre -1 e 1
    x = 0
    while(x < len(data)):
        data[x] = data[x]/maximo
        x += 1
    
    return data

A função separar_caracter recebe como entrada o Array correspondente ao captcha contendo 4 caracteres e o Array onde estão sendo armazenadas os caracteres separados. O está função realiza o processo de separar os caracteres do arquivo, remover o ruído contido no carácter e aplicar a função de normalização mostrada anteriormente, e por fim armazena os 4 segmentos corresponde aos caracteres em um Array que irá armazenar as saídas de todos os captchas de cada conjunto.

In [74]:
def separar_caracter(data, destination_array) :

    frame_lenght = 3150 # frames de fs/14 aprox 0.5s
    chars_values = list()
    
    #Separar Array data em 4 conjuntos 
    for audioPart in range(4):
        
        start = int(audioPart * len(data)/4)
        end = int((audioPart + 1)*len(data)/4)
        splited = data[start:end]
    
        initial_frame_pos = 0 
        frames_list = list()

        #Separando os frames
        while initial_frame_pos < len(splited):
            frames_list.append(splited[initial_frame_pos : initial_frame_pos+frame_lenght])
            initial_frame_pos += frame_lenght

        #Filtrando por amplitude maxima do frame inicial (supondo que é sempre ruido ou silencio) + desvio padrao
        amp_cat_values = list()

        max_values = [max(x) for x in frames_list] 
        max_values.sort()
        max_value_mean = np.mean(max_values)
        max_value_std = np.std(max_values)
       
        for frame in frames_list:
            frame_max = max(abs(frame))
            #se o frame tiver amplitude maxima <= a inicial std consideramos como um trecho de audio valido      
            if frame_max <= max_value_mean + max_value_std**2 :  
                if frame_max >= max_value_mean - max_value_std**2 :
                    amp_cat_values.append("max")
                else:
                    amp_cat_values.append("min")
            else:
                amp_cat_values.append("max")

        # guardando os indices dos frames de frames_list onde a amplitude equivale a um valor de audio valido
        max_indexes = list()
        index_list = range(0, len(amp_cat_values))
        amp_cat_values = list(zip(index_list, amp_cat_values))

        max_indexes = [index for index in amp_cat_values if index[1] is "max"]

        #agrupando por indices a uma distancia <= 2 posições (separando os caracteres)
        chars_indexes = list()
        new_frame = list()
        new_frame.append(max_indexes[0])
        for frame_info in max_indexes:
            if frame_info[0] - new_frame[-1][0] <= 2:
                if frame_info not in new_frame:
                    new_frame.append(frame_info)
            else:
                chars_indexes.append(new_frame)
                new_frame = list()
                new_frame.append(frame_info)
            if frame_info is max_indexes[-1]:
                chars_indexes.append(new_frame)

        # filtrando os indices no vetor de frames_list (contendo as amostras originais) para separar as amostras de
        # caracteres validos
        initialBigger = 0
        lastBigger = 0

        diferenca = 0

        for index in chars_indexes:
            initial = index[0][0] 
            last = index[-1][0] 

            if (last - initial) > diferenca:
                initialBigger = initial
                lastBigger = last

        if (initialBigger != 0):
            initialBigger = initialBigger - 1
    
        #Obter uma frequencia de corte baseada em um conjunto que possui apenas o ruído
        
        corte = 0
        if (initialBigger != 0):
            corte = max(frames_list[initialBigger - 1])
        elif (lastBigger + 2) < len(frames_list):     
            corte = max(frames_list[lastBigger + 2])
        elif (lastBigger + 1) < len(frames_list):     
            corte = max(frames_list[lastBigger + 1])
        else :
            corte = max(frames_list[lastBigger])
            
        value = np.hstack(frames_list[initialBigger:lastBigger+1])
               
        destination_array.append(normalizar(value))

Com as funções criadas, o próximo passo era aplicar a função por toda a base de treinamente. O trecho de código a seguir acessa a pasta da base de treinamento e aplica a função de separação de caracteres, além de separarmos o char correspondente a cada carácter do nome do arquivo e armazená-los no Array Y.

In [75]:
for root, dirs, files in os.walk("base_treinamento_I/"):  
    
    for filename in files:
        
        data, fs = librosa.load('base_treinamento_I/' + filename, None)
        for s in range(4):
            Y.append(filename[s])
            
        separar_caracter(data, WaveLetters)

for root, dirs, files in os.walk("base_treinamento_II/"):  
    
    for filename in files:
        
        data, fs = librosa.load('base_treinamento_II/' + filename, None)
        for s in range(4):
            Y.append(filename[s])
            
        separar_caracter(data, WaveLetters)

Após aplicarmos as transformações necessárias para remover a maior parte ruído do áudio em conjunto com a seguimentação dos caracteres realizada, a etapa seguinte foi o levantamento de possíveis atributos para utilizarmos nos algoritmos de classificação.

Dentre os atributos que utilizamos podemos citar a duração do áudio, batidas por minuto, média da frequência, média do centroide, quartis e média das frequências por segmento do áudio. Para a extração desses atributos criamos a função wave_attributes que será reapriveitada futuramente para extrair os atributos da base de validação. 

In [76]:
def wave_attributes(wave):

    #extraindo atributos
    duration = np.array([len(x)/fs for x in wave])

    tempo_beat = [librosa.beat.beat_track(y=x, sr=fs) for x in wave]
    tempo = np.array([x[0] for x in tempo_beat])
    beat_frame = np.array([x[1] for x in tempo_beat])

    mean_percussive = np.array([librosa.effects.hpss(x)[1].mean() for x in wave])

    fourier = [np.fft.fft(x) for x in wave]
    mean_freq = np.array([np.abs(x).mean() for x in fourier])

    mean_centroid = [librosa.feature.spectral_centroid(y=x, sr=fs).mean() for x in wave]

    freq_QT50 = [np.percentile(np.abs(x), 50) for x in fourier]
    freq_QT75 = [np.percentile(np.abs(x), 75) for x in fourier]
    freq_QT25 = [np.percentile(np.abs(x), 25) for x in fourier]

    #Médias
    mean_0_20 = []
    mean_20_40 = []
    mean_40_60 = []
    mean_60_80 = []
    mean_80_100 = []

    for x in wave:

        splited = []
        for i in range(5):
            start = int(i*len(x)/5)
            end = int((i+1)*len(x)/5)
            splited.append(x[start:end])

        mean_0_20.append(np.mean(np.absolute(splited[0])))
        mean_20_40.append(np.mean(np.absolute(splited[1])))
        mean_40_60.append(np.mean(np.absolute(splited[2])))
        mean_60_80.append(np.mean(np.absolute(splited[3])))
        mean_80_100.append(np.mean(np.absolute(splited[4])))
       
    return np.array(list(zip(
        duration,
        tempo, 
        mean_percussive, 
        mean_freq, 
        mean_centroid, 
        freq_QT50, 
        freq_QT75, 
        freq_QT25,
        mean_0_20,
        mean_20_40,
        mean_40_60,
        mean_60_80,
        mean_80_100)))

A seguir os atributos da base de treinamento serão extraidos e armazenado no array train_base

In [77]:
#extraindo atributos da base de treinamento
train_base = wave_attributes(WaveLetters) 

Após a obtenção dos atributos na base de treinamento, foram realizadas as mesmas etapas de segmentação e tratamento de ruídos na base de validação para que fosse possível obter os mesmos atributos que foram obtidos na base de treinamento.

In [78]:
#carregando base de validação
Array_test = list()
Y_test = list()

for root, dirs, files in os.walk("base_validacao_I/"):  
    
    for filename in files:
        
        data, fs = librosa.load('base_validacao_I/' + filename, None)
        for s in range(4):
            Y_test.append(filename[s])
            
        separar_caracter(data, Array_test)

for root, dirs, files in os.walk("base_validacao_II/"):  
    
    for filename in files:
        
        data, fs = librosa.load('base_validacao_II/' + filename, None)
        for s in range(4):
            Y_test.append(filename[s])
            
        separar_caracter(data, Array_test)

Extraindo atributos da base de validação e armazenando no vetor test_base

In [79]:
#extraindo atributos da base de treinamento
test_base = wave_attributes(Array_test) 

### 3 - Metodologia
Após a obtenção de possíveis atributos que poderiamos utilizar da nossa base de dados, realizamos uma análise para verificar qual ou quais os melhores atributos que poderiamos utilizar. Inicialmente separamos a nossa base de treinamento por carácter e geramos histogramas comparando os valores dos atributos obtidos em cada um desses caracteres para verificar se em algum destes atributos seria possível visualizar um padrão relevante.

In [80]:
#Criando bases de cada letra a,b,c,d,h,m,n,x,6,7

a_base = list()
b_base = list()
c_base = list()
d_base = list()
h_base = list()
m_base = list()
n_base = list()
x_base = list()
si_base = list()
se_base = list()
labels = [
    'duration',
    'tempo', 
    'mean_percussive', 
    'mean_freq', 
    'mean_centroid', 
    'freq_QT50', 
    'freq_QT75', 
    'freq_QT25',
    'mean1',
    'mean2',
    'mean3',
    'mean4',
    'mean5']

for i in range(0, len(Y)):
    if Y[i] == 'a':
        a_base.append(train_base[i])
    elif Y[i] == 'b':
        b_base.append(train_base[i])
    elif Y[i] == 'c':
        c_base.append(train_base[i])
    elif Y[i] == 'd':
        d_base.append(train_base[i])
    elif Y[i] == 'h':
        h_base.append(train_base[i])
    elif Y[i] == 'm':
        m_base.append(train_base[i])
    elif Y[i] == 'n':
        n_base.append(train_base[i])
    elif Y[i] == 'x':
        x_base.append(train_base[i])
    elif Y[i] == '6':
        si_base.append(train_base[i])
    elif Y[i] == '7':
        se_base.append(train_base[i])
    else:
        pass

a_base = pd.DataFrame.from_records(a_base, columns=labels)
b_base = pd.DataFrame.from_records(b_base, columns=labels)
c_base = pd.DataFrame.from_records(c_base, columns=labels)
d_base = pd.DataFrame.from_records(d_base, columns=labels)
h_base = pd.DataFrame.from_records(h_base, columns=labels)
m_base = pd.DataFrame.from_records(m_base, columns=labels)
n_base = pd.DataFrame.from_records(n_base, columns=labels)
x_base = pd.DataFrame.from_records(x_base, columns=labels)
si_base = pd.DataFrame.from_records(si_base, columns=labels)
se_base = pd.DataFrame.from_records(se_base, columns=labels)


#Base de teste

a_base_test = list()
b_base_test = list()
c_base_test = list()
d_base_test = list()
h_base_test = list()
m_base_test = list()
n_base_test = list()
x_base_test = list()
si_base_test = list()
se_base_test = list()

for i in range(0, len(Y_test)):
    if Y[i] == 'a':
        a_base_test.append(test_base[i])
    elif Y[i] == 'b':
        b_base_test.append(test_base[i])
    elif Y[i] == 'c':
        c_base_test.append(test_base[i])
    elif Y[i] == 'd':
        d_base_test.append(test_base[i])
    elif Y[i] == 'h':
        h_base_test.append(test_base[i])
    elif Y[i] == 'm':
        m_base_test.append(test_base[i])
    elif Y[i] == 'n':
        n_base_test.append(test_base[i])
    elif Y[i] == 'x':
        x_base_test.append(test_base[i])
    elif Y[i] == '6':
        si_base_test.append(test_base[i])
    elif Y[i] == '7':
        se_base_test.append(test_base[i])
    else:
        pass

a_base_test = pd.DataFrame.from_records(a_base_test, columns=labels)
b_base_test = pd.DataFrame.from_records(b_base_test, columns=labels)
c_base_test = pd.DataFrame.from_records(c_base_test, columns=labels)
d_base_test = pd.DataFrame.from_records(d_base_test, columns=labels)
h_base_test = pd.DataFrame.from_records(h_base_test, columns=labels)
m_base_test = pd.DataFrame.from_records(m_base_test, columns=labels)
n_base_test = pd.DataFrame.from_records(n_base_test, columns=labels)
x_base_test = pd.DataFrame.from_records(x_base_test, columns=labels)
si_base_test = pd.DataFrame.from_records(si_base_test, columns=labels)
se_base_test = pd.DataFrame.from_records(se_base_test, columns=labels)


Neste ponto, é criada uma função para a plotagem dos histogramas das features desejadas a partir do nome da feature

In [81]:
def plot_histogram(feature):
    output_notebook()

    a_hist, edges_a = np.histogram(getattr(a_base, feature), density=True)
    b_hist, edges_b = np.histogram(getattr(b_base, feature), density=True)
    c_hist, edges_c = np.histogram(getattr(c_base, feature), density=True)
    d_hist, edges_d = np.histogram(getattr(d_base, feature), density=True)
    h_hist, edges_h = np.histogram(getattr(h_base, feature), density=True)
    m_hist, edges_m = np.histogram(getattr(m_base, feature), density=True)
    n_hist, edges_n = np.histogram(getattr(n_base, feature), density=True)
    x_hist, edges_x = np.histogram(getattr(x_base, feature), density=True)
    si_hist, edges_si = np.histogram(getattr(si_base, feature), density=True)
    se_hist, edges_se = np.histogram(getattr(se_base, feature), density=True)

    fig_a_hist = figure(title="a_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_a_hist.quad(top=a_hist, bottom=0, left=edges_a[:-1], right=edges_a[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_b_hist = figure(title="b_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_b_hist.quad(top=b_hist, bottom=0, left=edges_b[:-1], right=edges_b[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_c_hist = figure(title="c_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_c_hist.quad(top=c_hist, bottom=0, left=edges_c[:-1], right=edges_c[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_d_hist = figure(title="d_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_d_hist.quad(top=d_hist, bottom=0, left=edges_d[:-1], right=edges_d[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_h_hist = figure(title="h_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_h_hist.quad(top=h_hist, bottom=0, left=edges_h[:-1], right=edges_h[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_m_hist = figure(title="m_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_m_hist.quad(top=m_hist, bottom=0, left=edges_m[:-1], right=edges_m[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_n_hist = figure(title="n_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_n_hist.quad(top=n_hist, bottom=0, left=edges_n[:-1], right=edges_n[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_x_hist = figure(title="x_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_x_hist.quad(top=x_hist, bottom=0, left=edges_x[:-1], right=edges_x[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_si_hist = figure(title="six_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_si_hist.quad(top=si_hist, bottom=0, left=edges_si[:-1], right=edges_si[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    fig_se_hist = figure(title="seven_"+feature, x_axis_label='value', y_axis_label='density', plot_width=250, plot_height=250)
    fig_se_hist.quad(top=se_hist, bottom=0, left=edges_se[:-1], right=edges_se[1:], fill_color="#036564", line_color="#033649", fill_alpha=0.3)

    plt = gridplot([[fig_a_hist, fig_b_hist, fig_c_hist], 
                    [fig_d_hist, fig_h_hist, fig_m_hist], 
                    [fig_n_hist, fig_x_hist, fig_si_hist], 
                    [fig_se_hist]], title=feature.upper(), notebook_handle=True)
    show(plt)

A primeira feature consultada foi a frequência média que foi obtida através da média do vetor de frequências obtidos pela transformada de fourie:

<b>Mean Frequency</b>

In [82]:
plot_histogram("mean_freq")

Loading BokehJS ...

Apesar de todos os caracteres demonstrarem ter uma faixa de frequência na qual a densidade é evidentemente mais alta do que as outras faixas, a densidade ainda não parece tão significativa a ponto de evidênciar a classe de cada caracter.
Vale notar que mesmo tendo perfis de distribuíção diferentes, os diversos caracteres apresentam alta densidade em faixas de frequencias muito semelhantes onde um possível sutíl agrupamento poderia ser formado como: {a, b, d, h, m, n};{n, x, 6};{c,7}.

Como a expectativa em cima da frequência média era alta em termos de relevância para o classificador, os quartis do vetor da transformada também foram considerados e avaliador, começando pelo primeiro quartil.

<b>QT25 Frequency</b>

In [83]:
plot_histogram("freq_QT25")

Loading BokehJS ...

Os histogramas desse quartil também não mostraram o QT25 como uma "killer feature" quanto ao nível de dissimilaridade entre os caracteres.

O quartil seguinte a ser analisado foi QT50

<b>QT50 Frequency</b>

In [84]:
plot_histogram("freq_QT50")

Loading BokehJS ...

O QT50 demonstrou que os caracteres possuem baixa densidade nessa faixa, com destaque para o caracter "a" que aparenta ter uma densidade maior do que os outros. Logo, essa feature parece um bom indicativo de carácter "a".

Por fim, o QT75 foi analisado

<b>QT75 Frequency</b>

In [85]:
plot_histogram("freq_QT75")

Loading BokehJS ...

Novamente, a densidade desta faixa parece menor do que a do QT25, e as faixas com maior densidade na feature estão muito próximas. Neste, os caracteres "c", "6" e "7" parecem de destacar dos demais pela faixa de frequência e tal semelhança também faz sentido com a pronúncia sonora próxima entre os três no início das palavras "SEis", "SEte" e "CE".

A próxima feature analisada foi o "Tempo" 

<b>Tempo</b>

In [86]:
plot_histogram("tempo")

Loading BokehJS ...

O Tempo também apresentou um resultado decepcionante quanto a qualidade de dissimilaridade entre os caracteres, apresentando a maioria com perfil de densidade muito parecido. Contudo os caracteres "x", "m", "n" e "7" demonstraram um perfil consideravelmente distinto dos outros, tornando o Tempo uma feature relevante para classificá-los.

Seguindo com a análise, a proxima feature considerada foi a média de percussão

<b>Mean Percussive</b>

In [87]:
plot_histogram("mean_percussive")

Loading BokehJS ...

Percussão média se apresentou com uma distribuição com aspecto gaussiano, na maioria dos caracteres com centro variando entre valores próximos a 0 com destaque para "c", "6" e "x" com as maiores densidades e  centro mais distantes de 0. Essa feature pareceu relevante para um corte, mas quase redundante com a frequência média.

Plotando-se então o centroide médio

<b>Mean centroid</b>

In [88]:
plot_histogram("mean_centroid")

Loading BokehJS ...

Os centróides não demostraram entregar um corte eficiente, mas agruparam por densidade maior em faixas semelhantes o "6", "x" e "b".

A partir deste ponto, foram avaliadas as médias de segmentos de 20% das amostras de amplitude ao longo do tempo geradas pela librosa.

<b> Mean 0-20</b>

In [89]:
plot_histogram("mean1")

Loading BokehJS ...

A primeira faixa de amostras analisadas apresentaram maior densidade entre 0 e 0.05.

<b> Mean 20-40</b>

In [90]:
plot_histogram("mean2")

Loading BokehJS ...

O segundo conjunto análisado se mostrou melhor do que o primeiro com perfis de densidade mais diversos e despontando como uma boa feature para classificação.

<b> Mean 40-60</b>

In [91]:
plot_histogram("mean3")

Loading BokehJS ...

A faixa acima apresentou uma forma mais "gaussiana" centrada entre 0.1 e 0.2 e com pouca dissimilaridade entre os caracteres.

<b>Mean 60-80</b>

In [92]:
plot_histogram("mean4")

Loading BokehJS ...

Novamente, as densidades se concentraram em faixas semelhantes com exceção ao "7" que teve pico com valores menores do que 0.05. Com isso é de se considerar a feature como um possível corte para este carácter.

<b> Mean 80-100</b>

In [93]:
plot_histogram("mean5")

Loading BokehJS ...

A última média avaliada tampouco apresentou novidades e ficou com densidades próximas concentradas abaixo de 0.1.

Após análisarmos os histogramas gerados, não foi possível identificar um atributo que sozinho pudesse ser relevante para utilizarmos na classificação de nossa base de validação, assim, foram plotados graficos dois a dois atributos com atributos que apresentaram frequências que poderiam ser diferenciativas mas em faixas comuns a mais de uma letra afim de se identificar se juntos poderiam gerar alguma aproximação melhor para uma função de classificação.

Primeiro a função para gerar o plot foi criada:

In [94]:
def plot_circle_density(feature_a, feature_b):
    fig = figure(x_axis_label=feature_a, y_axis_label=feature_b, title="Densidade "+feature_a+" x "+feature_b)
    fig.circle(x=getattr(a_base, feature_a), y=getattr(a_base, feature_b), color='#ffe6ee', legend='a')
    fig.circle(x=getattr(b_base, feature_a), y=getattr(b_base, feature_b), color='#b5e5fb', legend='b')
    fig.circle(x=getattr(c_base, feature_a), y=getattr(c_base, feature_b), color='#000000', legend='c')
    fig.circle(x=getattr(d_base, feature_a), y=getattr(d_base, feature_b), color='#5f37ab', legend='d')
    fig.circle(x=getattr(h_base, feature_a), y=getattr(h_base, feature_b), color='#298b95', legend='h')
    fig.circle(x=getattr(m_base, feature_a), y=getattr(m_base, feature_b), color='#561c22', legend='m')
    fig.circle(x=getattr(n_base, feature_a), y=getattr(n_base, feature_b), color='#610361', legend='n')
    fig.circle(x=getattr(x_base, feature_a), y=getattr(x_base, feature_b), color='#d82aac', legend='x')
    fig.circle(x=getattr(si_base, feature_a), y=getattr(si_base, feature_b), color='#bc8b79', legend='si')
    fig.circle(x=getattr(se_base, feature_a), y=getattr(se_base, feature_b), color='#6c819e', legend='se')
    show(fig)

Então foram plotados os gráficos com as features 2 a 2 que pareceram mais relevantes na etapa anterior (mean_freq, QT25, Tempo e Mean0-20) dois a dois de forma 

<b>Mean freq x QT25</b>

In [95]:
plot_circle_density("mean_freq", "freq_QT25")

O primeiro gráfico gerado teve um resultado frustrante devido à baixa visibilidade das classes já que se aglomeraram em uma mesma região e não demonstraram nenhuma forma de clusterização. 
Curioso notar também os outliers de todas as classes e a forma linear da distribuição dos pontos que provavelmente foi resultado do cruzamento de features relacionadas, ambas partem da frequência estimada pela transformada de fourier.

<b>Mean freq x Tempo</b>

In [96]:
plot_circle_density("mean_freq", "tempo")

O plot de Mean Freq x Tempo também deixou a desejar quanto a esperança de uma clusterização aparente. Mas chamou atenção para os valores constantes de Tempo dos caracteres. Infelizmente, os valores constantes são comuns a todos os caracteres, mas com a variação da frequência nesses valores constantes tal informação poderia ser útil.

<b>Mean freq x Mean 0-20</b>

In [97]:
plot_circle_density("mean_freq", "mean1")

Mean Freq X Mean 0-20 também não foi útil para uma distinção visual de um possível agrupamento, a distribuição também foi caótica e severamente sobreposta.


<b>QT25 x Mean 0-20</b>

In [98]:
plot_circle_density("freq_QT25", "mean1")

O resultado foi semelhante com a relação de QT25 e Mean 0-20. Um grande aglomerado com outliers esparsos.

<b>QT25 x Tempo</b>

In [99]:
plot_circle_density("freq_QT25", "tempo")

Novamente a curiosa característica quantizada do Tempo chama atenção no plot de QT25 x Tempo, mas infelizmente nenhum agrupamento visual para caracteres individuais pode ser detectado.

<b>Mean 0-20 x Tempo</b>

In [100]:
plot_circle_density("mean1", "tempo")

Por fim, a quantização do tempo também foi evidenciada no gráfico de Mean 0-20 x Tempo, com destaque a maior distribuição no eixo de Mean 0-20.

Verificado que de fato nenhuma dupla de atributos aparentemente poderia ser determinante para a classificação, foram realizados testes unindo os atributos de dois em dois e de três em três, além de utilizar um conjunto com todos os atributos juntos, e foram executados testes para verificar qual das combinações apresentava um melhor resultado.

Como os testes realizados foram um pouco longos, os testes que serão executados neste momento foram reduzidos apenas aos relevantes à análise anterior para se enquadrar no tempo de execução solicitado.

In [101]:
# Combinações de atributos para teste

freq_QT25_mean_percussive_train_base = np.array(list(zip(train_base[:,7], train_base[:,2])))
freq_QT25_mean_percussive_test_base = np.array(list(zip(test_base[:,7], test_base[:,2])))

freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,5], train_base[:,2])))
freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,5], test_base[:,2])))

freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,6], train_base[:,2])))
freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,6], test_base[:,2])))

tempo_mean_percussive_train_base = np.array(list(zip(train_base[:,1], train_base[:,2])))
tempo_mean_percussive_test_base = np.array(list(zip(test_base[:,1], test_base[:,2])))

mean_freq_mean_percussive_train_base = np.array(list(zip(train_base[:,3], train_base[:,2])))
mean_freq_mean_percussive_test_base = np.array(list(zip(test_base[:,3], test_base[:,2])))

mean_centroid_mean_percussive_train_base = np.array(list(zip(train_base[:,4], train_base[:,2])))
mean_centroid_mean_percussive_test_base = np.array(list(zip(test_base[:,4], test_base[:,2])))

mean_0_20_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,2])))
mean_0_20_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,2])))

mean_20_40_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,2])))
mean_20_40_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,2])))

mean_40_60_mean_percussive_train_base = np.array(list(zip(train_base[:,10], train_base[:,2])))
mean_40_60_mean_percussive_test_base = np.array(list(zip(test_base[:,10], test_base[:,2])))

mean_60_80_mean_percussive_train_base = np.array(list(zip(train_base[:,11], train_base[:,2])))
mean_60_80_mean_percussive_test_base = np.array(list(zip(test_base[:,11], test_base[:,2])))

mean_80_100_mean_percussive_train_base = np.array(list(zip(train_base[:,12], train_base[:,2])))
mean_80_100_mean_percussive_test_base = np.array(list(zip(test_base[:,12], test_base[:,2])))


tempo_mean_freq_train_base = np.array(list(zip(train_base[:,1], train_base[:,3])))
tempo_mean_freq_test_base = np.array(list(zip(test_base[:,1], test_base[:,3])))

mean_freq_mean_freq_train_base = np.array(list(zip(train_base[:,3], train_base[:,3])))
mean_freq_mean_freq_test_base = np.array(list(zip(test_base[:,3], test_base[:,3])))

mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,4], train_base[:,3])))
mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,4], test_base[:,3])))

freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,5], train_base[:,3])))
freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,5], test_base[:,3])))

freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,6], train_base[:,3])))
freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,6], test_base[:,3])))

freq_QT25_mean_freq_train_base = np.array(list(zip(train_base[:,7], train_base[:,3])))
freq_QT25_mean_freq_test_base = np.array(list(zip(test_base[:,7], test_base[:,3])))

mean_0_20_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,3])))
mean_0_20_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,3])))

mean_20_40_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,3])))
mean_20_40_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,3])))

mean_40_60_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,3])))
mean_40_60_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,3])))

mean_60_80_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,3])))
mean_60_80_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,3])))

mean_80_100_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,3])))
mean_80_100_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,3])))


tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,1], train_base[:,4])))
tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,1], test_base[:,4])))

freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,5], train_base[:,4])))
freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,5], test_base[:,4])))

freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,6], train_base[:,4])))
freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,6], test_base[:,4])))

freq_QT25_mean_centroid_train_base = np.array(list(zip(train_base[:,7], train_base[:,4])))
freq_QT25_mean_centroid_test_base = np.array(list(zip(test_base[:,7], test_base[:,4])))

mean_0_20_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,4])))
mean_0_20_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,4])))

mean_20_40_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,4])))
mean_20_40_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,4])))

mean_40_60_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,4])))
mean_40_60_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,4])))

mean_60_80_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,4])))
mean_60_80_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,4])))

mean_80_100_mean_centroid_train_base = np.array(list(zip(train_base[:,12], train_base[:,4])))
mean_80_100_mean_centroid_test_base = np.array(list(zip(test_base[:,12], test_base[:,4])))


freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,5], train_base[:,1])))
freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,5], test_base[:,1])))

freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,6], train_base[:,1])))
freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,6], test_base[:,1])))

freq_QT25_tempo_train_base = np.array(list(zip(train_base[:,7], train_base[:,1])))
freq_QT25_tempo_test_base = np.array(list(zip(test_base[:,7], test_base[:,1])))

mean_0_20_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,1])))
mean_0_20_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,1])))

mean_20_40_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,1])))
mean_20_40_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,1])))

mean_40_60_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,1])))
mean_40_60_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,1])))

mean_60_80_tempo_train_base = np.array(list(zip(train_base[:,11], train_base[:,1])))
mean_60_80_tempo_test_base = np.array(list(zip(test_base[:,11], test_base[:,1])))

mean_80_100_tempo_train_base = np.array(list(zip(train_base[:,12], train_base[:,1])))
mean_80_100_tempo_test_base = np.array(list(zip(test_base[:,12], test_base[:,1])))


freq_QT75_freq_QT50_train_base = np.array(list(zip(train_base[:,6], train_base[:,5])))
freq_QT75_freq_QT50_test_base = np.array(list(zip(test_base[:,6], test_base[:,5])))

freq_QT25_freq_QT50_train_base = np.array(list(zip(train_base[:,7], train_base[:,5])))
freq_QT25_freq_QT50_test_base = np.array(list(zip(test_base[:,7], test_base[:,5])))

mean_0_20_freq_QT50_train_base = np.array(list(zip(train_base[:,8], train_base[:,5])))
mean_0_20_freq_QT50_test_base = np.array(list(zip(test_base[:,8], test_base[:,5])))

mean_20_40_freq_QT50_train_base = np.array(list(zip(train_base[:,9], train_base[:,5])))
mean_20_40_freq_QT50_test_base = np.array(list(zip(test_base[:,9], test_base[:,5])))

mean_40_60_freq_QT50_train_base = np.array(list(zip(train_base[:,10], train_base[:,5])))
mean_40_60_freq_QT50_test_base = np.array(list(zip(test_base[:,10], test_base[:,5])))

mean_60_80_freq_QT50_train_base = np.array(list(zip(train_base[:,11], train_base[:,5])))
mean_60_80_freq_QT50_test_base = np.array(list(zip(test_base[:,11], test_base[:,5])))

mean_80_100_freq_QT50_train_base = np.array(list(zip(train_base[:,12], train_base[:,5])))
mean_80_100_freq_QT50_test_base = np.array(list(zip(test_base[:,12], test_base[:,5])))


freq_QT25_freq_QT75_train_base = np.array(list(zip(train_base[:,7], train_base[:,6])))
freq_QT25_freq_QT75_test_base = np.array(list(zip(test_base[:,7], test_base[:,6])))

mean_0_20_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,6])))
mean_0_20_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,6])))

mean_20_40_freq_QT75_train_base = np.array(list(zip(train_base[:,9], train_base[:,6])))
mean_20_40_freq_QT75_test_base = np.array(list(zip(test_base[:,9], test_base[:,6])))

mean_40_60_freq_QT75_train_base = np.array(list(zip(train_base[:,10], train_base[:,6])))
mean_40_60_freq_QT75_test_base = np.array(list(zip(test_base[:,10], test_base[:,6])))

mean_60_80_freq_QT75_train_base = np.array(list(zip(train_base[:,11], train_base[:,6])))
mean_60_80_freq_QT75_test_base = np.array(list(zip(test_base[:,11], test_base[:,6])))

mean_80_100_freq_QT75_train_base = np.array(list(zip(train_base[:,12], train_base[:,6])))
mean_80_100_freq_QT75_test_base = np.array(list(zip(test_base[:,12], test_base[:,6])))


mean_0_20_freq_QT25_train_base = np.array(list(zip(train_base[:,8], train_base[:,7])))
mean_0_20_freq_QT25_test_base = np.array(list(zip(test_base[:,8], test_base[:,7])))

mean_20_40_freq_QT25_train_base = np.array(list(zip(train_base[:,9], train_base[:,7])))
mean_20_40_freq_QT25_test_base = np.array(list(zip(test_base[:,9], test_base[:,7])))

mean_40_60_freq_QT25_train_base = np.array(list(zip(train_base[:,10], train_base[:,7])))
mean_40_60_freq_QT25_test_base = np.array(list(zip(test_base[:,10], test_base[:,7])))

mean_60_80_freq_QT25_train_base = np.array(list(zip(train_base[:,11], train_base[:,7])))
mean_60_80_freq_QT25_test_base = np.array(list(zip(test_base[:,11], test_base[:,7])))

mean_80_100_freq_QT25_train_base = np.array(list(zip(train_base[:,12], train_base[:,7])))
mean_80_100_freq_QT25_test_base = np.array(list(zip(test_base[:,12], test_base[:,7])))


mean_20_40_mean_0_20_train_base = np.array(list(zip(train_base[:,9], train_base[:,8])))
mean_20_40_mean_0_20_test_base = np.array(list(zip(test_base[:,9], test_base[:,8])))

mean_40_60_mean_0_20_train_base = np.array(list(zip(train_base[:,10], train_base[:,8])))
mean_40_60_mean_0_20_test_base = np.array(list(zip(test_base[:,10], test_base[:,8])))

mean_60_80_mean_0_20_train_base = np.array(list(zip(train_base[:,11], train_base[:,8])))
mean_60_80_mean_0_20_test_base = np.array(list(zip(test_base[:,11], test_base[:,8])))

mean_80_100_mean_0_20_train_base = np.array(list(zip(train_base[:,12], train_base[:,8])))
mean_80_100_mean_0_20_test_base = np.array(list(zip(test_base[:,12], test_base[:,8])))


mean_40_60_mean_20_40_train_base = np.array(list(zip(train_base[:,10], train_base[:,9])))
mean_40_60_mean_20_40_test_base = np.array(list(zip(test_base[:,10], test_base[:,9])))

mean_60_80_mean_20_40_train_base = np.array(list(zip(train_base[:,11], train_base[:,9])))
mean_60_80_mean_20_40_test_base = np.array(list(zip(test_base[:,11], test_base[:,9])))

mean_80_100_mean_20_40_train_base = np.array(list(zip(train_base[:,12], train_base[:,9])))
mean_80_100_mean_20_40_test_base = np.array(list(zip(test_base[:,12], test_base[:,9])))


mean_60_80_mean_40_60_train_base = np.array(list(zip(train_base[:,11], train_base[:,10])))
mean_60_80_mean_40_60_test_base = np.array(list(zip(test_base[:,11], test_base[:,10])))

mean_80_100_mean_40_60_train_base = np.array(list(zip(train_base[:,12], train_base[:,10])))
mean_80_100_mean_40_60_test_base = np.array(list(zip(test_base[:,12], test_base[:,10])))


mean_80_100_mean_60_80_train_base = np.array(list(zip(train_base[:,12], train_base[:,11])))
mean_80_100_mean_60_80_test_base = np.array(list(zip(test_base[:,12], test_base[:,11])))

#Conjunto de 3

mean_0_20_mean_20_40_mean_40_60_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,10])))
mean_0_20_mean_20_40_mean_40_60_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,10])))

mean_0_20_mean_20_40_mean_60_80_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,11])))
mean_0_20_mean_20_40_mean_60_80_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,11])))

mean_0_20_mean_20_40_mean_80_100_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,12])))
mean_0_20_mean_20_40_mean_80_100_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,12])))

mean_0_20_mean_20_40_freq_QT25_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,7])))
mean_0_20_mean_20_40_freq_QT25_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,7])))

mean_0_20_mean_20_40_freq_QT50_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,5])))
mean_0_20_mean_20_40_freq_QT50_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,5])))

mean_0_20_mean_20_40_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,6])))
mean_0_20_mean_20_40_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,6])))

mean_0_20_mean_20_40_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,2])))
mean_0_20_mean_20_40_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,2])))

mean_0_20_mean_20_40_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,1])))
mean_0_20_mean_20_40_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,1])))

mean_0_20_mean_20_40_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,4])))
mean_0_20_mean_20_40_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,4])))

mean_0_20_mean_20_40_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,9], train_base[:,3])))
mean_0_20_mean_20_40_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,9], test_base[:,3])))

mean_0_20_mean_40_60_mean_60_80_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,11])))
mean_0_20_mean_40_60_mean_60_80_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,11])))

mean_0_20_mean_40_60_mean_80_100_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,12])))
mean_0_20_mean_40_60_mean_80_100_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,12])))

mean_0_20_mean_40_60_freq_QT25_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,7])))
mean_0_20_mean_40_60_freq_QT25_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,7])))

mean_0_20_mean_40_60_freq_QT50_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,5])))
mean_0_20_mean_40_60_freq_QT50_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,5])))

mean_0_20_mean_40_60_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,6])))
mean_0_20_mean_40_60_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,6])))

mean_0_20_mean_40_60_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,2])))
mean_0_20_mean_40_60_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,2])))

mean_0_20_mean_40_60_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,1])))
mean_0_20_mean_40_60_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,1])))

mean_0_20_mean_40_60_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,4])))
mean_0_20_mean_40_60_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,4])))

mean_0_20_mean_40_60_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,10], train_base[:,3])))
mean_0_20_mean_40_60_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,10], test_base[:,3])))

mean_0_20_mean_60_80_mean_80_100_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,12])))
mean_0_20_mean_60_80_mean_80_100_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,12])))

mean_0_20_mean_60_80_freq_QT25_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,7])))
mean_0_20_mean_60_80_freq_QT25_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,7])))

mean_0_20_mean_60_80_freq_QT50_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,5])))
mean_0_20_mean_60_80_freq_QT50_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,5])))

mean_0_20_mean_60_80_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,6])))
mean_0_20_mean_60_80_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,6])))

mean_0_20_mean_60_80_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,2])))
mean_0_20_mean_60_80_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,2])))

mean_0_20_mean_60_80_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,1])))
mean_0_20_mean_60_80_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,1])))

mean_0_20_mean_60_80_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,4])))
mean_0_20_mean_60_80_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,4])))

mean_0_20_mean_60_80_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,11], train_base[:,3])))
mean_0_20_mean_60_80_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,11], test_base[:,3])))

mean_0_20_mean_80_100_freq_QT25_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,7])))
mean_0_20_mean_80_100_freq_QT25_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,7])))

mean_0_20_mean_80_100_freq_QT50_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,5])))
mean_0_20_mean_80_100_freq_QT50_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,5])))

mean_0_20_mean_80_100_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,6])))
mean_0_20_mean_80_100_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,6])))

mean_0_20_mean_80_100_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,2])))
mean_0_20_mean_80_100_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,2])))

mean_0_20_mean_80_100_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,1])))
mean_0_20_mean_80_100_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,1])))

mean_0_20_mean_80_100_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,4])))
mean_0_20_mean_80_100_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,4])))

mean_0_20_mean_80_100_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,12], train_base[:,3])))
mean_0_20_mean_80_100_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,12], test_base[:,3])))

mean_0_20_freq_QT25_freq_QT50_train_base = np.array(list(zip(train_base[:,8], train_base[:,7], train_base[:,5])))
mean_0_20_freq_QT25_freq_QT50_test_base = np.array(list(zip(test_base[:,8], test_base[:,7], test_base[:,5])))

mean_0_20_freq_QT25_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,7], train_base[:,6])))
mean_0_20_freq_QT25_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,7], test_base[:,6])))

mean_0_20_freq_QT25_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,7], train_base[:,2])))
mean_0_20_freq_QT25_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,7], test_base[:,2])))

mean_0_20_freq_QT25_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,7], train_base[:,1])))
mean_0_20_freq_QT25_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,7], test_base[:,1])))

mean_0_20_freq_QT25_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,7], train_base[:,4])))
mean_0_20_freq_QT25_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,7], test_base[:,4])))

mean_0_20_freq_QT25_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,7], train_base[:,3])))
mean_0_20_freq_QT25_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,7], test_base[:,3])))

mean_0_20_freq_QT50_freq_QT75_train_base = np.array(list(zip(train_base[:,8], train_base[:,5], train_base[:,6])))
mean_0_20_freq_QT50_freq_QT75_test_base = np.array(list(zip(test_base[:,8], test_base[:,5], test_base[:,6])))

mean_0_20_freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,5], train_base[:,2])))
mean_0_20_freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,5], test_base[:,2])))

mean_0_20_freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,5], train_base[:,1])))
mean_0_20_freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,5], test_base[:,1])))

mean_0_20_freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,5], train_base[:,4])))
mean_0_20_freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,5], test_base[:,4])))

mean_0_20_freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,5], train_base[:,3])))
mean_0_20_freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,5], test_base[:,3])))

mean_0_20_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,8], train_base[:,6], train_base[:,2])))
mean_0_20_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,8], test_base[:,6], test_base[:,2])))

mean_0_20_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,6], train_base[:,1])))
mean_0_20_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,6], test_base[:,1])))

mean_0_20_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,6], train_base[:,4])))
mean_0_20_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,6], test_base[:,4])))

mean_0_20_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,6], train_base[:,3])))
mean_0_20_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,6], test_base[:,3])))

mean_0_20_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,8], train_base[:,2], train_base[:,1])))
mean_0_20_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,8], test_base[:,2], test_base[:,1])))

mean_0_20_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,2], train_base[:,4])))
mean_0_20_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,2], test_base[:,4])))

mean_0_20_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,2], train_base[:,3])))
mean_0_20_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,2], test_base[:,3])))

mean_0_20_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,8], train_base[:,1], train_base[:,4])))
mean_0_20_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,8], test_base[:,1], test_base[:,4])))

mean_0_20_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,1], train_base[:,3])))
mean_0_20_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,1], test_base[:,3])))

mean_0_20_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,8], train_base[:,4], train_base[:,3])))
mean_0_20_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,8], test_base[:,4], test_base[:,3])))


mean_20_40_mean_40_60_mean_60_80_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,11])))
mean_20_40_mean_40_60_mean_60_80_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,11])))

mean_20_40_mean_40_60_mean_80_100_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,12])))
mean_20_40_mean_40_60_mean_80_100_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,12])))

mean_20_40_mean_40_60_freq_QT25_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,7])))
mean_20_40_mean_40_60_freq_QT25_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,7])))

mean_20_40_mean_40_60_freq_QT50_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,5])))
mean_20_40_mean_40_60_freq_QT50_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,5])))

mean_20_40_mean_40_60_freq_QT75_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,6])))
mean_20_40_mean_40_60_freq_QT75_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,6])))

mean_20_40_mean_40_60_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,2])))
mean_20_40_mean_40_60_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,2])))

mean_20_40_mean_40_60_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,1])))
mean_20_40_mean_40_60_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,1])))

mean_20_40_mean_40_60_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,4])))
mean_20_40_mean_40_60_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,4])))

mean_20_40_mean_40_60_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,10], train_base[:,3])))
mean_20_40_mean_40_60_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,10], test_base[:,3])))

mean_20_40_mean_60_80_mean_80_100_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,12])))
mean_20_40_mean_60_80_mean_80_100_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,12])))

mean_20_40_mean_60_80_freq_QT25_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,7])))
mean_20_40_mean_60_80_freq_QT25_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,7])))

mean_20_40_mean_60_80_freq_QT50_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,5])))
mean_20_40_mean_60_80_freq_QT50_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,5])))

mean_20_40_mean_60_80_freq_QT75_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,6])))
mean_20_40_mean_60_80_freq_QT75_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,6])))

mean_20_40_mean_60_80_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,2])))
mean_20_40_mean_60_80_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,2])))

mean_20_40_mean_60_80_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,1])))
mean_20_40_mean_60_80_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,1])))

mean_20_40_mean_60_80_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,4])))
mean_20_40_mean_60_80_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,4])))

mean_20_40_mean_60_80_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,11], train_base[:,3])))
mean_20_40_mean_60_80_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,11], test_base[:,3])))

mean_20_40_mean_80_100_freq_QT25_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,7])))
mean_20_40_mean_80_100_freq_QT25_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,7])))

mean_20_40_mean_80_100_freq_QT50_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,5])))
mean_20_40_mean_80_100_freq_QT50_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,5])))

mean_20_40_mean_80_100_freq_QT75_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,6])))
mean_20_40_mean_80_100_freq_QT75_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,6])))

mean_20_40_mean_80_100_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,2])))
mean_20_40_mean_80_100_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,2])))

mean_20_40_mean_80_100_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,1])))
mean_20_40_mean_80_100_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,1])))

mean_20_40_mean_80_100_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,4])))
mean_20_40_mean_80_100_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,4])))

mean_20_40_mean_80_100_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,12], train_base[:,3])))
mean_20_40_mean_80_100_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,12], test_base[:,3])))

mean_20_40_freq_QT25_freq_QT50_train_base = np.array(list(zip(train_base[:,9], train_base[:,7], train_base[:,5])))
mean_20_40_freq_QT25_freq_QT50_test_base = np.array(list(zip(test_base[:,9], test_base[:,7], test_base[:,5])))

mean_20_40_freq_QT25_freq_QT75_train_base = np.array(list(zip(train_base[:,9], train_base[:,7], train_base[:,6])))
mean_20_40_freq_QT25_freq_QT75_test_base = np.array(list(zip(test_base[:,9], test_base[:,7], test_base[:,6])))

mean_20_40_freq_QT25_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,7], train_base[:,2])))
mean_20_40_freq_QT25_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,7], test_base[:,2])))

mean_20_40_freq_QT25_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,7], train_base[:,1])))
mean_20_40_freq_QT25_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,7], test_base[:,1])))

mean_20_40_freq_QT25_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,7], train_base[:,4])))
mean_20_40_freq_QT25_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,7], test_base[:,4])))

mean_20_40_freq_QT25_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,7], train_base[:,3])))
mean_20_40_freq_QT25_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,7], test_base[:,3])))

mean_20_40_freq_QT50_freq_QT75_train_base = np.array(list(zip(train_base[:,9], train_base[:,5], train_base[:,6])))
mean_20_40_freq_QT50_freq_QT75_test_base = np.array(list(zip(test_base[:,9], test_base[:,5], test_base[:,6])))

mean_20_40_freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,5], train_base[:,2])))
mean_20_40_freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,5], test_base[:,2])))

mean_20_40_freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,5], train_base[:,1])))
mean_20_40_freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,5], test_base[:,1])))

mean_20_40_freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,5], train_base[:,4])))
mean_20_40_freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,5], test_base[:,4])))

mean_20_40_freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,5], train_base[:,3])))
mean_20_40_freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,5], test_base[:,3])))

mean_20_40_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,9], train_base[:,6], train_base[:,2])))
mean_20_40_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,9], test_base[:,6], test_base[:,2])))

mean_20_40_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,6], train_base[:,1])))
mean_20_40_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,6], test_base[:,1])))

mean_20_40_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,6], train_base[:,4])))
mean_20_40_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,6], test_base[:,4])))

mean_20_40_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,6], train_base[:,3])))
mean_20_40_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,6], test_base[:,3])))

mean_20_40_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,9], train_base[:,2], train_base[:,1])))
mean_20_40_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,9], test_base[:,2], test_base[:,1])))

mean_20_40_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,2], train_base[:,4])))
mean_20_40_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,2], test_base[:,4])))

mean_20_40_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,2], train_base[:,3])))
mean_20_40_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,2], test_base[:,3])))

mean_20_40_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,9], train_base[:,1], train_base[:,4])))
mean_20_40_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,9], test_base[:,1], test_base[:,4])))

mean_20_40_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,1], train_base[:,3])))
mean_20_40_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,1], test_base[:,3])))

mean_20_40_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,9], train_base[:,4], train_base[:,3])))
mean_20_40_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,9], test_base[:,4], test_base[:,3])))


mean_40_60_mean_60_80_freq_QT25_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,7])))
mean_40_60_mean_60_80_freq_QT25_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,7])))

mean_40_60_mean_60_80_freq_QT50_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,5])))
mean_40_60_mean_60_80_freq_QT50_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,5])))

mean_40_60_mean_60_80_freq_QT75_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,6])))
mean_40_60_mean_60_80_freq_QT75_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,6])))

mean_40_60_mean_60_80_mean_percussive_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,2])))
mean_40_60_mean_60_80_mean_percussive_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,2])))

mean_40_60_mean_60_80_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,1])))
mean_40_60_mean_60_80_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,1])))

mean_40_60_mean_60_80_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,4])))
mean_40_60_mean_60_80_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,4])))

mean_40_60_mean_60_80_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,11], train_base[:,3])))
mean_40_60_mean_60_80_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,11], test_base[:,3])))

mean_40_60_mean_80_100_freq_QT25_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,7])))
mean_40_60_mean_80_100_freq_QT25_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,7])))

mean_40_60_mean_80_100_freq_QT50_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,5])))
mean_40_60_mean_80_100_freq_QT50_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,5])))

mean_40_60_mean_80_100_freq_QT75_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,6])))
mean_40_60_mean_80_100_freq_QT75_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,6])))

mean_40_60_mean_80_100_mean_percussive_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,2])))
mean_40_60_mean_80_100_mean_percussive_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,2])))

mean_40_60_mean_80_100_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,1])))
mean_40_60_mean_80_100_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,1])))

mean_40_60_mean_80_100_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,4])))
mean_40_60_mean_80_100_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,4])))

mean_40_60_mean_80_100_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,12], train_base[:,3])))
mean_40_60_mean_80_100_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,12], test_base[:,3])))

mean_40_60_freq_QT25_freq_QT50_train_base = np.array(list(zip(train_base[:,10], train_base[:,7], train_base[:,5])))
mean_40_60_freq_QT25_freq_QT50_test_base = np.array(list(zip(test_base[:,10], test_base[:,7], test_base[:,5])))

mean_40_60_freq_QT25_freq_QT75_train_base = np.array(list(zip(train_base[:,10], train_base[:,7], train_base[:,6])))
mean_40_60_freq_QT25_freq_QT75_test_base = np.array(list(zip(test_base[:,10], test_base[:,7], test_base[:,6])))

mean_40_60_freq_QT25_mean_percussive_train_base = np.array(list(zip(train_base[:,10], train_base[:,7], train_base[:,2])))
mean_40_60_freq_QT25_mean_percussive_test_base = np.array(list(zip(test_base[:,10], test_base[:,7], test_base[:,2])))

mean_40_60_freq_QT25_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,7], train_base[:,1])))
mean_40_60_freq_QT25_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,7], test_base[:,1])))

mean_40_60_freq_QT25_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,7], train_base[:,4])))
mean_40_60_freq_QT25_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,7], test_base[:,4])))

mean_40_60_freq_QT25_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,7], train_base[:,3])))
mean_40_60_freq_QT25_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,7], test_base[:,3])))

mean_40_60_freq_QT50_freq_QT75_train_base = np.array(list(zip(train_base[:,10], train_base[:,5], train_base[:,6])))
mean_40_60_freq_QT50_freq_QT75_test_base = np.array(list(zip(test_base[:,10], test_base[:,5], test_base[:,6])))

mean_40_60_freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,10], train_base[:,5], train_base[:,2])))
mean_40_60_freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,10], test_base[:,5], test_base[:,2])))

mean_40_60_freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,5], train_base[:,1])))
mean_40_60_freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,5], test_base[:,1])))

mean_40_60_freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,5], train_base[:,4])))
mean_40_60_freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,5], test_base[:,4])))

mean_40_60_freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,5], train_base[:,3])))
mean_40_60_freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,5], test_base[:,3])))

mean_40_60_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,10], train_base[:,6], train_base[:,2])))
mean_40_60_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,10], test_base[:,6], test_base[:,2])))

mean_40_60_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,6], train_base[:,1])))
mean_40_60_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,6], test_base[:,1])))

mean_40_60_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,6], train_base[:,4])))
mean_40_60_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,6], test_base[:,4])))

mean_40_60_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,6], train_base[:,3])))
mean_40_60_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,6], test_base[:,3])))

mean_40_60_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,10], train_base[:,2], train_base[:,1])))
mean_40_60_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,10], test_base[:,2], test_base[:,1])))

mean_40_60_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,2], train_base[:,4])))
mean_40_60_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,2], test_base[:,4])))

mean_40_60_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,2], train_base[:,3])))
mean_40_60_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,2], test_base[:,3])))

mean_40_60_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,10], train_base[:,1], train_base[:,4])))
mean_40_60_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,10], test_base[:,1], test_base[:,4])))

mean_40_60_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,1], train_base[:,3])))
mean_40_60_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,1], test_base[:,3])))

mean_40_60_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,10], train_base[:,4], train_base[:,3])))
mean_40_60_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,10], test_base[:,4], test_base[:,3])))


mean_60_80_mean_80_100_freq_QT25_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,7])))
mean_60_80_mean_80_100_freq_QT25_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,7])))

mean_60_80_mean_80_100_freq_QT50_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,5])))
mean_60_80_mean_80_100_freq_QT50_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,5])))

mean_60_80_mean_80_100_freq_QT75_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,6])))
mean_60_80_mean_80_100_freq_QT75_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,6])))

mean_60_80_mean_80_100_mean_percussive_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,2])))
mean_60_80_mean_80_100_mean_percussive_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,2])))

mean_60_80_mean_80_100_tempo_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,1])))
mean_60_80_mean_80_100_tempo_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,1])))

mean_60_80_mean_80_100_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,4])))
mean_60_80_mean_80_100_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,4])))

mean_60_80_mean_80_100_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,12], train_base[:,3])))
mean_60_80_mean_80_100_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,12], test_base[:,3])))

mean_60_80_freq_QT25_freq_QT50_train_base = np.array(list(zip(train_base[:,11], train_base[:,7], train_base[:,5])))
mean_60_80_freq_QT25_freq_QT50_test_base = np.array(list(zip(test_base[:,11], test_base[:,7], test_base[:,5])))

mean_60_80_freq_QT25_freq_QT75_train_base = np.array(list(zip(train_base[:,11], train_base[:,7], train_base[:,6])))
mean_60_80_freq_QT25_freq_QT75_test_base = np.array(list(zip(test_base[:,11], test_base[:,7], test_base[:,6])))

mean_60_80_freq_QT25_mean_percussive_train_base = np.array(list(zip(train_base[:,11], train_base[:,7], train_base[:,2])))
mean_60_80_freq_QT25_mean_percussive_test_base = np.array(list(zip(test_base[:,11], test_base[:,7], test_base[:,2])))

mean_60_80_freq_QT25_tempo_train_base = np.array(list(zip(train_base[:,11], train_base[:,7], train_base[:,1])))
mean_60_80_freq_QT25_tempo_test_base = np.array(list(zip(test_base[:,11], test_base[:,7], test_base[:,1])))

mean_60_80_freq_QT25_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,7], train_base[:,4])))
mean_60_80_freq_QT25_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,7], test_base[:,4])))

mean_60_80_freq_QT25_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,7], train_base[:,3])))
mean_60_80_freq_QT25_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,7], test_base[:,3])))

mean_60_80_freq_QT50_freq_QT75_train_base = np.array(list(zip(train_base[:,11], train_base[:,5], train_base[:,6])))
mean_60_80_freq_QT50_freq_QT75_test_base = np.array(list(zip(test_base[:,11], test_base[:,5], test_base[:,6])))

mean_60_80_freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,11], train_base[:,5], train_base[:,2])))
mean_60_80_freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,11], test_base[:,5], test_base[:,2])))

mean_60_80_freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,11], train_base[:,5], train_base[:,1])))
mean_60_80_freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,11], test_base[:,5], test_base[:,1])))

mean_60_80_freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,5], train_base[:,4])))
mean_60_80_freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,5], test_base[:,4])))

mean_60_80_freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,5], train_base[:,3])))
mean_60_80_freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,5], test_base[:,3])))

mean_60_80_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,11], train_base[:,6], train_base[:,2])))
mean_60_80_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,11], test_base[:,6], test_base[:,2])))

mean_60_80_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,11], train_base[:,6], train_base[:,1])))
mean_60_80_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,11], test_base[:,6], test_base[:,1])))

mean_60_80_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,6], train_base[:,4])))
mean_60_80_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,6], test_base[:,4])))

mean_60_80_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,6], train_base[:,3])))
mean_60_80_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,6], test_base[:,3])))

mean_60_80_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,11], train_base[:,2], train_base[:,1])))
mean_60_80_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,11], test_base[:,2], test_base[:,1])))

mean_60_80_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,2], train_base[:,4])))
mean_60_80_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,2], test_base[:,4])))

mean_60_80_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,2], train_base[:,3])))
mean_60_80_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,2], test_base[:,3])))

mean_60_80_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,11], train_base[:,1], train_base[:,4])))
mean_60_80_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,11], test_base[:,1], test_base[:,4])))

mean_60_80_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,1], train_base[:,3])))
mean_60_80_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,1], test_base[:,3])))

mean_60_80_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,11], train_base[:,4], train_base[:,3])))
mean_60_80_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,11], test_base[:,4], test_base[:,3])))


mean_80_100_freq_QT25_freq_QT50_train_base = np.array(list(zip(train_base[:,12], train_base[:,7], train_base[:,5])))
mean_80_100_freq_QT25_freq_QT50_test_base = np.array(list(zip(test_base[:,12], test_base[:,7], test_base[:,5])))

mean_80_100_freq_QT25_freq_QT75_train_base = np.array(list(zip(train_base[:,12], train_base[:,7], train_base[:,6])))
mean_80_100_freq_QT25_freq_QT75_test_base = np.array(list(zip(test_base[:,12], test_base[:,7], test_base[:,6])))

mean_80_100_freq_QT25_mean_percussive_train_base = np.array(list(zip(train_base[:,12], train_base[:,7], train_base[:,2])))
mean_80_100_freq_QT25_mean_percussive_test_base = np.array(list(zip(test_base[:,12], test_base[:,7], test_base[:,2])))

mean_80_100_freq_QT25_tempo_train_base = np.array(list(zip(train_base[:,12], train_base[:,7], train_base[:,1])))
mean_80_100_freq_QT25_tempo_test_base = np.array(list(zip(test_base[:,12], test_base[:,7], test_base[:,1])))

mean_80_100_freq_QT25_mean_centroid_train_base = np.array(list(zip(train_base[:,12], train_base[:,7], train_base[:,4])))
mean_80_100_freq_QT25_mean_centroid_test_base = np.array(list(zip(test_base[:,12], test_base[:,7], test_base[:,4])))

mean_80_100_freq_QT25_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,7], train_base[:,3])))
mean_80_100_freq_QT25_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,7], test_base[:,3])))

mean_80_100_freq_QT50_freq_QT75_train_base = np.array(list(zip(train_base[:,12], train_base[:,5], train_base[:,6])))
mean_80_100_freq_QT50_freq_QT75_test_base = np.array(list(zip(test_base[:,12], test_base[:,5], test_base[:,6])))

mean_80_100_freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,12], train_base[:,5], train_base[:,2])))
mean_80_100_freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,12], test_base[:,5], test_base[:,2])))

mean_80_100_freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,12], train_base[:,5], train_base[:,1])))
mean_80_100_freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,12], test_base[:,5], test_base[:,1])))

mean_80_100_freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,12], train_base[:,5], train_base[:,4])))
mean_80_100_freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,12], test_base[:,5], test_base[:,4])))

mean_80_100_freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,5], train_base[:,3])))
mean_80_100_freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,5], test_base[:,3])))

mean_80_100_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,12], train_base[:,6], train_base[:,2])))
mean_80_100_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,12], test_base[:,6], test_base[:,2])))

mean_80_100_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,12], train_base[:,6], train_base[:,1])))
mean_80_100_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,12], test_base[:,6], test_base[:,1])))

mean_80_100_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,12], train_base[:,6], train_base[:,4])))
mean_80_100_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,12], test_base[:,6], test_base[:,4])))

mean_80_100_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,6], train_base[:,3])))
mean_80_100_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,6], test_base[:,3])))

mean_80_100_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,12], train_base[:,2], train_base[:,1])))
mean_80_100_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,12], test_base[:,2], test_base[:,1])))

mean_80_100_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,12], train_base[:,2], train_base[:,4])))
mean_80_100_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,12], test_base[:,2], test_base[:,4])))

mean_80_100_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,2], train_base[:,3])))
mean_80_100_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,2], test_base[:,3])))

mean_80_100_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,12], train_base[:,1], train_base[:,4])))
mean_80_100_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,12], test_base[:,1], test_base[:,4])))

mean_80_100_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,1], train_base[:,3])))
mean_80_100_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,1], test_base[:,3])))

mean_80_100_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,12], train_base[:,4], train_base[:,3])))
mean_80_100_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,12], test_base[:,4], test_base[:,3])))


freq_QT25_freq_QT50_freq_QT75_train_base = np.array(list(zip(train_base[:,7], train_base[:,5], train_base[:,6])))
freq_QT25_freq_QT50_freq_QT75_test_base = np.array(list(zip(test_base[:,7], test_base[:,5], test_base[:,6])))

freq_QT25_freq_QT50_mean_percussive_train_base = np.array(list(zip(train_base[:,7], train_base[:,5], train_base[:,2])))
freq_QT25_freq_QT50_mean_percussive_test_base = np.array(list(zip(test_base[:,7], test_base[:,5], test_base[:,2])))

freq_QT25_freq_QT50_tempo_train_base = np.array(list(zip(train_base[:,7], train_base[:,5], train_base[:,1])))
freq_QT25_freq_QT50_tempo_test_base = np.array(list(zip(test_base[:,7], test_base[:,5], test_base[:,1])))

freq_QT25_freq_QT50_mean_centroid_train_base = np.array(list(zip(train_base[:,7], train_base[:,5], train_base[:,4])))
freq_QT25_freq_QT50_mean_centroid_test_base = np.array(list(zip(test_base[:,7], test_base[:,5], test_base[:,4])))

freq_QT25_freq_QT50_mean_freq_train_base = np.array(list(zip(train_base[:,7], train_base[:,5], train_base[:,3])))
freq_QT25_freq_QT50_mean_freq_test_base = np.array(list(zip(test_base[:,7], test_base[:,5], test_base[:,3])))

freq_QT25_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,7], train_base[:,6], train_base[:,2])))
freq_QT25_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,7], test_base[:,6], test_base[:,2])))

freq_QT25_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,7], train_base[:,6], train_base[:,1])))
freq_QT25_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,7], test_base[:,6], test_base[:,1])))

freq_QT25_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,7], train_base[:,6], train_base[:,4])))
freq_QT25_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,7], test_base[:,6], test_base[:,4])))

freq_QT25_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,7], train_base[:,6], train_base[:,3])))
freq_QT25_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,7], test_base[:,6], test_base[:,3])))

freq_QT25_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,7], train_base[:,2], train_base[:,1])))
freq_QT25_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,7], test_base[:,2], test_base[:,1])))

freq_QT25_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,7], train_base[:,2], train_base[:,4])))
freq_QT25_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,7], test_base[:,2], test_base[:,4])))

freq_QT25_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,7], train_base[:,2], train_base[:,3])))
freq_QT25_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,7], test_base[:,2], test_base[:,3])))

freq_QT25_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,7], train_base[:,1], train_base[:,4])))
freq_QT25_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,7], test_base[:,1], test_base[:,4])))

freq_QT25_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,7], train_base[:,1], train_base[:,3])))
freq_QT25_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,7], test_base[:,1], test_base[:,3])))

freq_QT25_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,7], train_base[:,4], train_base[:,3])))
freq_QT25_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,7], test_base[:,4], test_base[:,3])))

freq_QT50_freq_QT75_mean_percussive_train_base = np.array(list(zip(train_base[:,5], train_base[:,6], train_base[:,2])))
freq_QT50_freq_QT75_mean_percussive_test_base = np.array(list(zip(test_base[:,5], test_base[:,6], test_base[:,2])))

freq_QT50_freq_QT75_tempo_train_base = np.array(list(zip(train_base[:,5], train_base[:,6], train_base[:,1])))
freq_QT50_freq_QT75_tempo_test_base = np.array(list(zip(test_base[:,5], test_base[:,6], test_base[:,1])))

freq_QT50_freq_QT75_mean_centroid_train_base = np.array(list(zip(train_base[:,5], train_base[:,6], train_base[:,4])))
freq_QT50_freq_QT75_mean_centroid_test_base = np.array(list(zip(test_base[:,5], test_base[:,6], test_base[:,4])))

freq_QT50_freq_QT75_mean_freq_train_base = np.array(list(zip(train_base[:,5], train_base[:,6], train_base[:,3])))
freq_QT50_freq_QT75_mean_freq_test_base = np.array(list(zip(test_base[:,5], test_base[:,6], test_base[:,3])))

freq_QT50_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,5], train_base[:,2], train_base[:,1])))
freq_QT50_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,5], test_base[:,2], test_base[:,1])))

freq_QT50_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,5], train_base[:,2], train_base[:,4])))
freq_QT50_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,5], test_base[:,2], test_base[:,4])))

freq_QT50_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,5], train_base[:,2], train_base[:,3])))
freq_QT50_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,5], test_base[:,2], test_base[:,3])))

freq_QT50_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,5], train_base[:,1], train_base[:,4])))
freq_QT50_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,5], test_base[:,1], test_base[:,4])))

freq_QT50_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,5], train_base[:,1], train_base[:,3])))
freq_QT50_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,5], test_base[:,1], test_base[:,3])))

freq_QT50_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,5], train_base[:,4], train_base[:,3])))
freq_QT50_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,5], test_base[:,4], test_base[:,3])))

freq_QT75_mean_percussive_tempo_train_base = np.array(list(zip(train_base[:,6], train_base[:,2], train_base[:,1])))
freq_QT75_mean_percussive_tempo_test_base = np.array(list(zip(test_base[:,6], test_base[:,2], test_base[:,1])))

freq_QT75_mean_percussive_mean_centroid_train_base = np.array(list(zip(train_base[:,6], train_base[:,2], train_base[:,4])))
freq_QT75_mean_percussive_mean_centroid_test_base = np.array(list(zip(test_base[:,6], test_base[:,2], test_base[:,4])))

freq_QT75_mean_percussive_mean_freq_train_base = np.array(list(zip(train_base[:,6], train_base[:,2], train_base[:,3])))
freq_QT75_mean_percussive_mean_freq_test_base = np.array(list(zip(test_base[:,6], test_base[:,2], test_base[:,3])))

freq_QT75_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,6], train_base[:,1], train_base[:,4])))
freq_QT75_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,6], test_base[:,1], test_base[:,4])))

freq_QT75_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,6], train_base[:,1], train_base[:,3])))
freq_QT75_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,6], test_base[:,1], test_base[:,3])))

freq_QT75_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,6], train_base[:,4], train_base[:,3])))
freq_QT75_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,6], test_base[:,4], test_base[:,3])))

mean_percussive_tempo_mean_centroid_train_base = np.array(list(zip(train_base[:,2], train_base[:,1], train_base[:,4])))
mean_percussive_tempo_mean_centroid_test_base = np.array(list(zip(test_base[:,2], test_base[:,1], test_base[:,4])))

mean_percussive_tempo_mean_freq_train_base = np.array(list(zip(train_base[:,2], train_base[:,1], train_base[:,3])))
mean_percussive_tempo_mean_freq_test_base = np.array(list(zip(test_base[:,2], test_base[:,1], test_base[:,3])))

mean_percussive_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,2], train_base[:,4], train_base[:,3])))
mean_percussive_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,2], test_base[:,4], test_base[:,3])))

tempo_mean_centroid_mean_freq_train_base = np.array(list(zip(train_base[:,1], train_base[:,4], train_base[:,3])))
tempo_mean_centroid_mean_freq_test_base = np.array(list(zip(test_base[:,1], test_base[:,4], test_base[:,3])))


train_vector = [
    train_base,
    freq_QT25_tempo_mean_freq_train_base, 
    mean_0_20_freq_QT25_mean_freq_train_base, 
    mean_0_20_freq_QT25_tempo_train_base,
    mean_0_20_mean_freq_train_base, 
    tempo_mean_freq_train_base,  
    freq_QT25_mean_freq_train_base, 
    freq_QT50_freq_QT75_mean_percussive_train_base, 
    freq_QT75_mean_percussive_tempo_train_base,
    mean_percussive_tempo_mean_centroid_train_base, 
    tempo_mean_centroid_mean_freq_train_base]

test_vector = [
    test_base,
    freq_QT25_tempo_mean_freq_test_base, 
    mean_0_20_freq_QT25_mean_freq_test_base, 
    mean_0_20_freq_QT25_tempo_test_base,
    mean_0_20_mean_freq_test_base, 
    tempo_mean_freq_test_base, 
    freq_QT25_mean_freq_test_base, 
    freq_QT50_freq_QT75_mean_percussive_test_base, 
    freq_QT75_mean_percussive_tempo_test_base,
    mean_percussive_tempo_mean_centroid_test_base,
    tempo_mean_centroid_mean_freq_test_base]

power_test_labels = [
    "test_base",
    "freq_QT25_tempo_mean_freq",
    "mean_0_20_freq_QT25_mean_freq",
    "mean_0_20_freq_QT25_tempo",
    "mean_0_20_mean_freq",
    "tempo_mean_freq",
    "freq_QT25_mean_freq",
    "freq_QT50_freq_QT75_mean_percussive",
    "freq_QT75_mean_percussive_tempo",
    "mean_percussive_tempo_mean_centroid",
    "tempo_mean_centroid_mean_freq_test_base"]

No mesmo teste que foi realizado para verificar qual o melhor conjunto de variavéis para o processo de classificação, testamos também diversos algoritmos de classificação aplicando os algoritmos na base de treinamento e validação para obter qual o algoritmo apresentava o melhor resultado para conjunto de variavéis. Para este teste foi considerado apenas o número de caracteres corretamente classificados, e não o número de captchas que tiveram todos os caracteres classificados corretamente.

In [102]:
#power_test
def power_classifier(train, test):
    
    #TREINANDO
    
    # Neural Net
    
    neu = MLPClassifier()
    neu.fit(train, Y)
      
    # Random Forest
    
    rnf = RandomForestClassifier()
    rnf.fit(train, Y)
    
    # Gaussian Naive-Bayes
    
    gnb = GaussianNB()
    gnb.fit(train, Y)
    
    #Discriminante Linear
    
    clf = LinearDiscriminantAnalysis()
    clf.fit(train, Y)
    
    #KNN
    norm_train = preprocessing.normalize(train, norm='l1')
    norm_test = preprocessing.normalize(test, norm='l1')
    
    neigh1 = KNeighborsClassifier(n_neighbors=1)
    neigh1.fit(norm_train, Y)
    
    neigh3 = KNeighborsClassifier(n_neighbors=3)
    neigh3.fit(norm_train, Y)

    neigh5 = KNeighborsClassifier(n_neighbors=5)
    neigh5.fit(norm_train, Y)

    
    #DECISION TREE

    dtr = tree.DecisionTreeClassifier()
    dtr = dtr.fit(train, Y)
    


    clf_false = 0
    clf_true = 0

    #TESTANDO

    #Discriminante Linear
    for x in range(0, len(test)):
        if clf.predict([test[x]])[0] == Y_test[x]:
            clf_true += 1
        else:
            clf_false += 1

    #KNN
    neigh1_false = 0
    neigh1_true = 0
    for x in range(0, len(test)):
        if neigh1.predict([norm_test[x]])[0] == Y_test[x]:
            neigh1_true += 1
        else:
            neigh1_false += 1

    neigh3_false = 0
    neigh3_true = 0
    for x in range(0, len(test)):
        if neigh3.predict([norm_test[x]])[0] == Y_test[x]:
            neigh3_true += 1
        else:
            neigh3_false += 1
            
    neigh5_false = 0
    neigh5_true = 0
    for x in range(0, len(test)):
        if neigh5.predict([norm_test[x]])[0] == Y_test[x]:
            neigh5_true += 1
        else:
            neigh5_false += 1
            
    #DECISION TREE
            
    dtr_false = 0
    dtr_true = 0
    for x in range(0, len(test)):
        if dtr.predict([test[x]])[0] == Y_test[x]:
            dtr_true += 1
        else:
            dtr_false += 1
            
    #Random Forest
            
    rnf_false = 0
    rnf_true = 0
    for x in range(0, len(test)):
        if rnf.predict([test[x]])[0] == Y_test[x]:
            rnf_true += 1
        else:
            rnf_false += 1
            
    #Neural Net
            
    neu_false = 0
    neu_true = 0
    for x in range(0, len(test)):
        if neu.predict([test[x]])[0] == Y_test[x]:
            neu_true += 1
        else:
            neu_false += 1
            
    # Gaussian Naive-Bayes

    gnb_false = 0
    gnb_true = 0
    for x in range(0, len(test)):
        if gnb.predict([test[x]])[0] == Y_test[x]:
            gnb_true += 1
        else:
            gnb_false += 1            

           
    rnf_diff = rnf_true - rnf_false
    neigh1_diff = neigh1_true - neigh1_false
    neigh3_diff = neigh3_true - neigh3_false
    neigh5_diff = neigh5_true - neigh5_false
    dtr_diff = dtr_true - dtr_false
    clf_diff = clf_true - clf_false
    gnb_diff = gnb_true - gnb_false
    neu_diff = neu_true - neu_false

  

    best_method = max([('neigh1_diff', neigh1_diff), 
                       ('neigh3_diff', neigh3_diff),
                       ('neigh5_diff', neigh5_diff), 
                       ('dtr_diff', dtr_diff),
                       ('clf_diff', clf_diff),
                       ('gnb_diff', gnb_diff),
                       ('neu_diff', neu_diff),
                       ('rnf_diff', rnf_diff),],
                     key=lambda item:item[1])
    
    return(best_method)

Para auxiliar nos testes, foi criada a função power_test que exibe a diferença de quantos caracteres o algoritmo conseguiu classificar corretamente e quantos foram classificados errados e qual foi o algoritmo de classificação que obteve este resultado. Ao final é exibido a base de dado e o algoritmo que obtiveram um melhor desempenho.

In [103]:
def power_test():
    best_result = list()
    for x in range(0, len(train_vector)):
        pclss = power_classifier(train_vector[x], test_vector[x])
        print("Base: " + power_test_labels[x] +" Result: " + str(pclss))
        best_result.append((power_test_labels[x], pclss[0], pclss[1]))

        
    print(str(len(test_base)) + " ELEMENTOS TESTADOS\n RESULTADOS")
        
    best_result.sort(reverse=True, key=lambda item:item[2])
    
    print("Os metodos ordenados são: "+ str(best_result))
    
    best_result = max(best_result, key=lambda item:item[2])
    
    
    print("A Melhor combinação é: " + str(best_result))

    return 0

Podemos verificar pela execução do algoritmo abaixo, que o conjunto de dados que obteve um melhor resultado foi o que continha todos os atributos gerados, e o algoritmo que obteve tal resultado foi o algoritmo de Random Forest. Desta forma estes são respectivamente o conjunto de atributos e o algoritmo de classificação que utilizaremos em nosso projeto.

In [104]:
power_test()

Base: test_base Result: ('rnf_diff', -12)
Base: freq_QT25_tempo_mean_freq Result: ('dtr_diff', -482)
Base: mean_0_20_freq_QT25_mean_freq Result: ('rnf_diff', -636)
Base: mean_0_20_freq_QT25_tempo Result: ('rnf_diff', -636)
Base: mean_0_20_mean_freq Result: ('rnf_diff', -742)
Base: tempo_mean_freq Result: ('dtr_diff', -588)
Base: freq_QT25_mean_freq Result: ('rnf_diff', -600)
Base: freq_QT50_freq_QT75_mean_percussive Result: ('rnf_diff', -498)
Base: freq_QT75_mean_percussive_tempo Result: ('rnf_diff', -550)
Base: mean_percussive_tempo_mean_centroid Result: ('rnf_diff', -572)
Base: tempo_mean_centroid_mean_freq_test_base Result: ('rnf_diff', -458)
1184 ELEMENTOS TESTADOS
 RESULTADOS
Os metodos ordenados são: [('test_base', 'rnf_diff', -12), ('tempo_mean_centroid_mean_freq_test_base', 'rnf_diff', -458), ('freq_QT25_tempo_mean_freq', 'dtr_diff', -482), ('freq_QT50_freq_QT75_mean_percussive', 'rnf_diff', -498), ('freq_QT75_mean_percussive_tempo', 'rnf_diff', -550), ('mean_percussive_tempo_m

0

Foram avaliadas tambem algumas técnicas de clusterização como o K-Means e o AgglomerativeClustering com diferentes criterios de união dos grupos com dissimilaridade euclidiana. Os resultados podem ser vistos abaixo:

In [105]:
def print_groups(in_classifier):
    
    d = {'a':[], 'b':[], 'c':[], 'd':[], 'h':[], 'm':[], 'n':[], 'x':[], '6':[], '7':[]}

    for i in in_classifier:
        d[i[0]].append(i[1])
    print("a: " + str(scipy.stats.mode(d['a']).mode))
    print("b: " + str(scipy.stats.mode(d['b']).mode))
    print("c: " + str(scipy.stats.mode(d['c']).mode))
    print("d: " + str(scipy.stats.mode(d['d']).mode))
    print("h: " + str(scipy.stats.mode(d['h']).mode))
    print("m: " + str(scipy.stats.mode(d['m']).mode))
    print("n: " + str(scipy.stats.mode(d['n']).mode))
    print("x: " + str(scipy.stats.mode(d['x']).mode))
    print("6: " + str(scipy.stats.mode(d['6']).mode))
    print("7: " + str(scipy.stats.mode(d['7']).mode))

<b>K-means</b>

In [106]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(train_base)

v = list(zip(Y_test, kmeans.predict(test_base)))
print_groups(v)

a: [4]
b: [0]
c: [2]
d: [1]
h: [0]
m: [8]
n: [9]
x: [1]
6: [2]
7: [8]


O K-Means não foi capaz de criar um cluster único por caracter com base no train_base. Provavelmente exista muita sobreposição na distribuição dos dados.

<b>AgglomerativeClustering</b>

In [107]:
agg_ward = AgglomerativeClustering(n_clusters=10, linkage="ward")

v = list(zip(Y_test, agg_ward.fit_predict(train_base, test_base)))
print_groups(v)

a: [4]
b: [0]
c: [0]
d: [0]
h: [0]
m: [0]
n: [2]
x: [0]
6: [0]
7: [0]


In [108]:
agg_ward = AgglomerativeClustering(n_clusters=10, linkage="complete")

v = list(zip(Y_test, agg_ward.fit_predict(train_base, test_base)))
print_groups(v)

a: [0]
b: [0]
c: [0]
d: [0]
h: [0]
m: [0]
n: [0]
x: [0]
6: [0]
7: [0]


In [109]:
agg_ward = AgglomerativeClustering(n_clusters=10, linkage="average")

v = list(zip(Y_test, agg_ward.fit_predict(train_base, test_base)))
print_groups(v)

a: [1]
b: [1]
c: [1]
d: [1]
h: [1]
m: [1]
n: [1]
x: [1]
6: [1]
7: [1]


Semelhante ao K-means, o AgglomerativeClustering não obteve um grupo único por caracter, dessa forma os dois algoritmos de clusterização necessitam de um maior refinamento e possivelmente uma combinação entre si para gerar uma clusterização satisfatória correndo sérios riscos de um overfit na base.


Após a definição do algoritmo que iriamos utilizar para classificar nosso conjunto, identificamos que para sementes diferentes que eram dadas ao algoritmo de classificação, retornavam resultados diferentes. Desta forma testamos um grande número de sementes para verificar qual delas retornava um melhor resultado. Como métrica para este teste, verificamos a porcentagem de captchas que foram corretamente classificados. Como foram utilizadas diversas sementes para este teste, o código abaixo irá apenas reproduzir uma pequena amostragem, com o objetivo de diminuir o tempo de execução total do projeto.

In [110]:
rnf_best_result = 0
best_seed = 0

for i in range(2300, 2400):
    rnf_false = 0
    rnf_true = 0
    test_result = 0
    
    rnf = RandomForestClassifier(random_state=i)
    rnf.fit(train_base, Y)
    
    #Predizer array de letras
    Y_predict = rnf.predict(test_base)
    
    #Inicializar captchas de letras real e predict
    captch_predict = []
    captch_real = []

    x = 0
    while (x < len(Y_predict)):
        captch_predict.append(Y_predict[x] + Y_predict[x+1] + Y_predict[x+2] + Y_predict[x+3])
        captch_real.append(Y_test[x] + Y_test[x+1] + Y_test[x+2] + Y_test[x+3])
        x = x+4

    #Calcular acurácia do captcha
    test_result = accuracy_score(captch_predict,captch_real)

    if test_result > rnf_best_result:
        rnf_best_result = test_result
        best_seed = i
    
       
print("Treinamento finalizado.")
print("O melhor resultado foi: " + str(rnf_best_result))
print("A melhor semente foi: " + str(best_seed))

Treinamento finalizado.
O melhor resultado foi: 0.11148648648648649
A melhor semente foi: 2332


### 4 - Resultados

Agora que obtemos a melhor semente para a execução do Random Forest, iremos verificar a acurácia dos caracteres e a acurácia dos captcha.

In [111]:
rnf_false = 0
rnf_true = 0
test_result = 0
    
rnf_false_caracter = 0
rnf_true_caracter = 0
test_result_caracter = 0

rnf = RandomForestClassifier(random_state=best_seed)
rnf.fit(train_base, Y)
 
#Predizer array de letras
Y_predict = rnf.predict(test_base)
    
#Inicializar captchas de letras real e predict
captch_predict = []
captch_real = []

x = 0
while (x < len(Y_predict)):
    captch_predict.append(Y_predict[x] + Y_predict[x+1] + Y_predict[x+2] + Y_predict[x+3])
    captch_real.append(Y_test[x] + Y_test[x+1] + Y_test[x+2] + Y_test[x+3])
    x = x+4

#Calcular acurácia
test_result_captcha = accuracy_score(captch_predict,captch_real)   
test_result_caracter = accuracy_score(Y_predict,Y_test)                

print("TOTAL DE CARACTERES CLASSIFICADOS: " +  str(len(Y_predict)))
print("ACERTOS CARACTERES: " +  str(round((len(Y_predict) * test_result_caracter), 0)))
print("ERROS CARACTERES: " + str(len(Y_predict) - round((len(Y_predict) * test_result_caracter), 0)))
print("ACURACIA CARACTERES: " + str(test_result_caracter * 100) + "% \n\n")

print("TOTAL DE CAPTCHAS CLASSIFICADOS: " +  str(len(captch_predict)))
print("ACERTOS CAPTCHAS: " +  str(round((len(captch_predict) * test_result_captcha), 0)))
print("ERROS CAPTCHAS: " + str(len(captch_predict) - round((len(captch_predict) * test_result_captcha), 0)))
print("ACURACIA CAPTCHAS: " + str(test_result_captcha * 100) + "%")

TOTAL DE CARACTERES CLASSIFICADOS: 1184
ACERTOS CARACTERES: 611.0
ERROS CARACTERES: 573.0
ACURACIA CARACTERES: 51.604729729729726% 


TOTAL DE CAPTCHAS CLASSIFICADOS: 296
ACERTOS CAPTCHAS: 33.0
ERROS CAPTCHAS: 263.0
ACURACIA CAPTCHAS: 11.14864864864865%


### 5 -  Comentários Finais

Os resultados obtidos não foram de forma alguma satisfatórios, pois a acurácia da classificação das strings completas foi consideravelmente baixa. Entretanto, é notável que a acurácia da classificação de caracteres individuais é bem superior.
De fato, a escolha dos atributos junto com a baixa qualidade das gravações gerou uma distribuição de dados que não pode ser compreendida de maneira satisfatoriamente constante por nenhum dos métodos testados.
Assim, as medidas a serem tomadas seriam a obtenção de um conjunto de atributos mais expressivo e/ou a escolha de um método de classificação diferente dos testados que possa se adaptar melhor a distribuição do problema, além disso um método que conseguisse seguimentar os aúdios de maneira mais satisfatória poderia  também colaborar para uma melhora no resultado final.
